In [1]:
import requests,json,random, sqlite3
import pandas as pd
from hashlib import sha1
from time import time
from keys import headers

url = "https://restcountries-v1.p.rapidapi.com/all"

def regions_api_rest(url, headers):
    result = requests.request("GET", url, headers=headers).json()

    without_region =[]
    regions = []
    
    for item in result:
     #Bouvet island and Mcdonald Island  >> 'N/A'
    	if item['region']=='':
        	item['region'] = 'N/A'
        	without_region.append(item)

    	if item['region'] not in regions:
        	regions.append(item['region'])
    return regions, without_region


def country_regions(regions, without_region):
    reg =[]
    city = [] 
    lang = []
    time_ejec = []
    
    df = pd.DataFrame(columns=('Region', 'City name', 'Language', 'Time(ms)'), index=None)
    
    if isinstance(regions, list):
        for region in regions:
            if isinstance(region, str):
            
                if region != 'N/A':
                    t_i = time()
                    url = "https://restcountries.eu/rest/v2/region/" + region
                    countries = requests.request("GET", url).json()
                    
                    
                    country_select = random.choice(countries)
                
                    for lang_data in country_select['languages']:
                        language_name = sha1(lang_data['name'].encode('utf-8')).hexdigest()
                    reg.append(region)
                    city.append(country_select['name'])
                    lang.append(language_name) 
                    t_f = time()
                    time_ejec.append(float("%.2f" % ((t_f-t_i)*1000)))
                elif region == 'N/A':
                    t_i2 = time()
                    country_select = random.choice(without_region)
                    #bouvet island >>N/A
                    if country_select['languages']== []:
                        country_select['languages'] = 'N/A'
                    
                    #encrypts
                    for lang_data in country_select['languages']:
                        language_name = sha1(lang_data.encode('utf-8')).hexdigest()
                    reg.append(region)
                    city.append(country_select['name'])
                    lang.append(language_name)
                    t_f2 = time()
                    time_ejec.append(float("%.2f" % ((t_f2-t_i2)*1000)))
                    df['Region'] = reg
                    df['City name'] = city
                    df['Language'] = lang
                    df['Time(ms)'] = time_ejec
                    #6 times
                    time_max = df['Time(ms)'].max()
                    time_min = df['Time(ms)'].min()
                    time_total = df['Time(ms)'].sum()
                    time_average = df['Time(ms)'].mean()
                    return df, time_max, time_min, time_total, time_average
                else:
                    print('No es una opcion')
            else:
                print('Tipo de dato no valido: {0}. ingrese un valor tipo String.'.format(type(op)))
    else:
        print('ingrese la lista de regiones')
        
        
def sqlite_insert(table_data):
    conn = sqlite3.connect('resultado_Prueba.db')
    table_data.to_sql(name='Tabla_Regiones', con=conn, if_exists='replace')
    table_data.to_json('data.json')

In [2]:
[regions, without_region] = regions_api_rest(url, headers)
[table_data, time_max, time_min, time_total, time_average] = country_regions(regions, without_region)
#7.Save Result and 8 generate Json
sqlite_insert(table_data)

/Users/JuanMunoz/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2612: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [3]:
table_data

,Region,City name,Language,Time(ms)
0,Asia,Nepal,16e340746126ed50b9b41c11e75938dcc3f78df2,398.26
1,Europe,Ukraine,515451058797b204e188dd70f8d20f32dacca327,361.28
2,Africa,Mozambique,23882c575954a0789bf02aba9e6dd01f539bc738,372.47
3,Oceania,Tokelau,649df08a448ee3fa90f3746baaf6b0907df42c91,385.06
4,Americas,Virgin Islands (British),649df08a448ee3fa90f3746baaf6b0907df42c91,386.34
5,Polar,Antarctica,74ad93cbebf587ac46ab0ef1e3f24139ddd3f535,336.55
6,N/A,Heard Island and McDonald Islands,094b0fe0e302854af1311afab85b5203ba457a3b,0.01


In [4]:
print(f"tiempo total {time_total:.2f}\ntiempo promedio {time_average:.2f}\ntiempo minimo {time_min:.2f}\ntiempo maximo {time_max:.2f}")


tiempo total 2239.97
tiempo promedio 320.00
tiempo minimo 0.01
tiempo maximo 398.26
